Let's see how we can use an LLM to try and predict the edgelist of a graph just from the 7 features given to us.

!pip install transformers accelerate flash_attn peft datasets bitsandbytes trl pandas

In [1]:
import torch

from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, PeftModelForCausalLM, AutoPeftModelForCausalLM
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    set_seed,
    pipeline,
    DataCollatorForLanguageModeling
)
from trl import SFTTrainer
     

In [2]:
if torch.cuda.is_bf16_supported():
  compute_dtype = torch.bfloat16
  attn_implementation = 'flash_attention_2'
else:
  compute_dtype = torch.float16
  attn_implementation = 'sdpa'

device_map = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(attn_implementation, ',', device_map)

flash_attention_2 , cuda


In [3]:
# Model configuration
model_id = "microsoft/phi-3-mini-4k-instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

# Load model and tokenizer with explicit device map
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": torch.cuda.current_device()},  # This is the key change
    trust_remote_code=True,
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model.config.use_cache = False  # Required for gradient checkpointing

messages = [{"role": "user", "content": "Do you know what a graph edgelist is ? Answer in a single short sentence."}]

inputs = tokenizer.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt").to('cuda')

outputs = model.generate(inputs, max_new_tokens=32)

text = tokenizer.batch_decode(outputs)[0]

print(text)

## Onto preparting the way for training

We generated the data in `generate_LLM_sets.ipynb`.

In [5]:
from generate_LLM_sets import ConcatenatedFileDataset

In [6]:
train = torch.load('data/train.pt')
val = torch.load('data/valid.pt')

/tmp/ipykernel_63583/748146594.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train = torch.load('data/train.pt')
/tmp/ipykernel_63583/748146594.py:2: FutureWarning: Yo

In [7]:
print(train[0])

{'prompt': 'Give the graph edgelist associated to the following features.-Number of nodes: 15.0-Number of edges: 105.0-Average degree: 14.0-Number of triangles: 455.0-Clustering coefficient: 1.0-Max k cores: 14.0-Number of communities: 1.0', 'answer': '(0, 1), (0, 14), (0, 2), (0, 13), (0, 3), (0, 12), (0, 4), (0, 11), (0, 5), (0, 10), (0, 6), (0, 9), (0, 7), (0, 8), (1, 2), (1, 3), (1, 14), (1, 4), (1, 13), (1, 5), (1, 12), (1, 6), (1, 11), (1, 7), (1, 10), (1, 8), (1, 9), (14, 2), (14, 3), (14, 4), (14, 5), (14, 6), (14, 7), (14, 8), (14, 9), (14, 10), (14, 11), (14, 12), (14, 13), (2, 3), (2, 4), (2, 5), (2, 6), (2, 13), (2, 7), (2, 12), (2, 8), (2, 11), (2, 9), (2, 10), (13, 3), (13, 4), (13, 5), (13, 6), (13, 7), (13, 8), (13, 9), (13, 10), (13, 11), (13, 12), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 12), (3, 10), (3, 11), (12, 4), (12, 5), (12, 6), (12, 7), (12, 8), (12, 9), (12, 10), (12, 11), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (4, 11), (11, 5), (11, 6),

Adapt to template


In [7]:
def create_message_column(row):
    messages = [
        {"content": row['prompt'], "role": "user"},
        {"content": row['answer'], "role": "assistant"}
    ]
    return {"messages": messages}

def format_dataset_chatml(row):
    # Apply the chat template
    formatted_text = tokenizer.apply_chat_template(
        row["messages"], 
        add_generation_prompt=False, 
        tokenize=False
    )
    
    # Tokenize the text
    tokenized = tokenizer(
        formatted_text,
        truncation=True,
        max_length=4096,
        padding=False,
        return_tensors="pt",  # Return PyTorch tensors (or omit for lists)
    )
    
    return {
        "input_ids": tokenized["input_ids"][0],  # Assuming batch size of 1 for simplicity
        "attention_mask": tokenized["attention_mask"][0]  # Include this if needed
    }

In [8]:
# Prepare datasets with batched processing
train_dataset = train.map(create_message_column)
train_dataset = train_dataset.map(format_dataset_chatml)

val_dataset = val.map(create_message_column)
val_dataset = val_dataset.map(format_dataset_chatml)

del train
del val

torch.save(train_dataset, 'data/train_dataset_tokenized.pt')
torch.save(val_dataset, 'data/val_dataset_tokenized.pt')

In [9]:
train_dataset = torch.load('data/train_dataset_tokenized.pt')
val_dataset = torch.load('data/val_dataset_tokenized.pt')


/tmp/ipykernel_8765/1450608261.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_dataset = torch.load('data/train_dataset_tokenized.pt')
/tmp/ipykernel_8765/14506082

In [10]:
print("Sample from training dataset:")
print(list(train_dataset[0].keys()))  # Should include 'input_ids'
print("Length of first sample:", len(train_dataset[0]['input_ids']))

Sample from training dataset:
['input_ids', 'attention_mask']
Length of first sample: 788


## Let's go


In [11]:
# LoRA Configuration
lora_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    inference_mode=False,
)

# Prepare model for training
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.config.use_cache = False
model.gradient_checkpointing_enable()

In [12]:
training_args = TrainingArguments(
    output_dir="./phi-3-mini-LoRA",
    evaluation_strategy="steps",
    do_eval=True,
    max_steps=-1,
    optim="adamw_torch",
    per_device_train_batch_size=1,  # Reduced from 8
    gradient_accumulation_steps=16,   # Increased from 4
    per_device_eval_batch_size=1,    # Reduced from 8
    log_level="info",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=1e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    eval_steps=125,
    num_train_epochs=1,
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    seed=0,
    gradient_checkpointing=True,
    report_to="tensorboard",
)

/home/marceau/anaconda3/envs/altegrad/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, 
    mlm=False,
)

The next cell is the most cursed thing known to man but hey it fixes all my problems and I have been fighting this notebook for way too long now.

In [14]:
def patched_forward(self, *args, **kwargs):
    kwargs.pop("num_items_in_batch", None)  # Ignore the extra argument
    return PeftModelForCausalLM.forward(self, *args, **kwargs)  # Replace 'MyModelClass' with your model's actual class name

from types import MethodType

model.forward = MethodType(patched_forward, model)

In [15]:
# Initialize trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    peft_config=lora_config,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,  # Added data collator
)

# Training
trainer.train()

# Save the final model
trainer.save_model()

# Optional: Save adapter only
model.save_pretrained("./phi-3-mini-LoRA/final_adapter")

merged_model = model.merge_and_unload()
save_path = "./merged_model"
merged_model.save_pretrained(save_path)

/tmp/ipykernel_8765/1639492350.py:2: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(
/home/marceau/anaconda3/envs/altegrad/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/marceau/anaconda3/envs/altegrad/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
Using auto half precision backend
***** Running training *****
  Num examples = 8,000
  Num Epochs = 1
  Instantaneous batch size per 

  0%|          | 0/500 [00:00<?, ?it/s]

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.
/home/marceau/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


{'loss': 9.2236, 'grad_norm': 3.1693050861358643, 'learning_rate': 2e-05, 'epoch': 0.02}
{'loss': 8.739, 'grad_norm': 2.2582690715789795, 'learning_rate': 4e-05, 'epoch': 0.04}
{'loss': 8.2193, 'grad_norm': 1.3784129619598389, 'learning_rate': 6e-05, 'epoch': 0.06}
{'loss': 7.0056, 'grad_norm': 1.6875430345535278, 'learning_rate': 8e-05, 'epoch': 0.08}
{'loss': 5.6848, 'grad_norm': 4.614770889282227, 'learning_rate': 0.0001, 'epoch': 0.1}
{'loss': 4.2678, 'grad_norm': 1.5896835327148438, 'learning_rate': 9.987820251299122e-05, 'epoch': 0.12}
{'loss': 3.9761, 'grad_norm': 0.992117702960968, 'learning_rate': 9.951340343707852e-05, 'epoch': 0.14}
{'loss': 3.5681, 'grad_norm': 1.0812567472457886, 'learning_rate': 9.890738003669029e-05, 'epoch': 0.16}
{'loss': 3.6988, 'grad_norm': 1.237465739250183, 'learning_rate': 9.806308479691595e-05, 'epoch': 0.18}
{'loss': 3.8249, 'grad_norm': 1.1787524223327637, 'learning_rate': 9.698463103929542e-05, 'epoch': 0.2}
{'loss': 3.7028, 'grad_norm': 1.521


***** Running Evaluation *****
  Num examples = 1000
  Batch size = 1


...

Saving model checkpoint to ./phi-3-mini-LoRA/checkpoint-500


{'eval_loss': 0.2023552656173706, 'eval_runtime': 877.7394, 'eval_samples_per_second': 1.139, 'eval_steps_per_second': 1.139, 'epoch': 1.0}


loading configuration file config.json from cache at /home/marceau/.cache/huggingface/hub/models--microsoft--phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,

{'train_runtime': 26707.2465, 'train_samples_per_second': 0.3, 'train_steps_per_second': 0.019, 'train_loss': 3.806950183868408, 'epoch': 1.0}


loading configuration file config.json from cache at /home/marceau/.cache/huggingface/hub/models--microsoft--phi-3-mini-4k-instruct/snapshots/0a67737cc96d2554230f90338b163bc6380a2a85/config.json
Model config Phi3Config {
  "_name_or_path": "Phi-3-mini-4k-instruct",
  "architectures": [
    "Phi3ForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "auto_map": {
    "AutoConfig": "microsoft/phi-3-mini-4k-instruct--configuration_phi3.Phi3Config",
    "AutoModelForCausalLM": "microsoft/phi-3-mini-4k-instruct--modeling_phi3.Phi3ForCausalLM"
  },
  "bos_token_id": 1,
  "embd_pdrop": 0.0,
  "eos_token_id": 32000,
  "hidden_act": "silu",
  "hidden_size": 3072,
  "initializer_range": 0.02,
  "intermediate_size": 8192,
  "max_position_embeddings": 4096,
  "model_type": "phi3",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 32,
  "original_max_position_embeddings": 4096,
  "pad_token_id": 32000,
  "resid_pdrop": 0.0,
  "rms_norm_eps": 1e-05,